Start of sysnonym code, the algorithm is as following: We use lead-k for the text extractive summerization (note that we are discarding the abstract part of the paper), and then use the sysnoyms to make the generated text more lay.


In [ ]:
import pandas as pd
from zipfile import ZipFile

# Load dataset
def loadDataset(path, fileName):
  zip_file = ZipFile(path)
  return pd.read_json(zip_file.open(fileName))


The following code is for lead k algorithm, which takes the top k sentences from each section of the paper, and combine them together

In [ ]:
!pip3 install wordfreq
from wordfreq import word_frequency

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.9 MB/s eta 0:00:00


The following code is for synonyms and hypernum replacement of the origianl text, we are using those replacement to make our summery more lay

In [ ]:
import nltk
from nltk.corpus import wordnet, stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from wordfreq import zipf_frequency
import string


nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

def replace_uncommon_words(text):
    if not isinstance(text, str):
        text = str(text)

    # Tokenize the text
    words = word_tokenize(text)
    
    # Get common English words from NLTK's stopwords
    common_words = set(stopwords.words('english'))

    
    # Define a function to get the most frequent synonym using WordNet
    def get_most_frequent_synonym(word):
        synonyms = []
        for syn in wordnet.synsets(word):
            for lemma in syn.lemmas():
                synonyms.append(lemma.name())
        if not synonyms:
            return word
        synonyms_freq = [(syn, zipf_frequency(syn, 'en')) for syn in synonyms]
        # print([word,max(synonyms_freq, key=lambda x: x[1])[0],synonyms,synonyms_freq])
        return max(synonyms_freq, key=lambda x: x[1])[0]

    
    def replace_with_hypernym(word):
      synsets = wordnet.synsets(word)

      if synsets:
          hypernyms = synsets[0].hypernyms()

          if hypernyms:
              hypernym_word = hypernyms[0].lemmas()[0].name()
              return hypernym_word
          else:
              return word
      else:
          return word

    # check if there's number in string
    def has_numbers(inputString):
      return any(char.isdigit() for char in inputString)

    # Iterate over words and replace uncommon words with the most frequent synonym
    new_words = []
    for word in words:
        if word.strip() not in string.punctuation and word not in common_words and not has_numbers(word) and 1 < zipf_frequency(word, 'en') < 3.5:
            new_word = get_most_frequent_synonym(word)
            newZipfScore = zipf_frequency(new_word, 'en') 
            if newZipfScore < 2:
              new_word = replace_with_hypernym(new_word)
              # print(f"new word not Work! replace with {new_word}")
            new_words.append(new_word)
        else:
            new_words.append(word)

    # Join the words back into a single string
    new_text = ' '.join(new_words)
    return new_text



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
def newLeadK(paper, **kwargs):
  k = kwargs.get('k', 1)  # Default value of k is 1
  extractive_Sum = []
  # id = 0
  section_series = paper["sections"]
  section_arr = section_series.values
  for section in section_arr[0]:
    for i in range(min(k,len(section))):
      sentence = section[i]
      modified_sentence = replace_uncommon_words(sentence)
      extractive_Sum.append(modified_sentence)
  return [extractive_Sum]

In [ ]:
import json
import os
import pickle
import glob

In [ ]:
# the following code will get the latest pickle file in the pickles folder
def most_recent_pickle_index(save_folder_path):
  filenames = os.listdir(save_folder_path)

  # Filter filenames that match the pattern "checkpoint_{index}.pkl"
  filtered_filenames = [filename for filename in filenames if filename.startswith('checkpoint_') and filename.endswith('.pkl')]

  # Extract the indices from the filenames and convert them to integers
  indices = [int(filename.split('_')[1].split('.')[0]) for filename in filtered_filenames]

  # Check if there are any filenames
  if indices:
      # Find the filename with the highest index
      return max(indices)
  else:
      return None


In [ ]:
def summarize_with_checkpoint(path, JSONfileName, summary_function, nameOfYourSummary, save_folder_path, **kwargs):
  '''
  path: path to original dataset
  JSONfileName: name of original dataset
  summary_function: takes a (1, n) row of the original dataset as input, returns a summary string
  nameOfYourSummary: the name to save the new summary under in the final dataframe
  

  '''
  recent_index = most_recent_pickle_index(save_folder_path)
  print(f'recent index is {recent_index}')
  recent_path = f'{save_folder_path}checkpoint_{recent_index}.pkl'
  
  original_dataset = loadDataset(path, JSONfileName)

  if recent_index == None:
    #first time in:
    our_summary = pd.DataFrame()
    #set up the correct colums
    cols_to_copy = ['id', 'year', 'title', 'abstract', 'summary', 'keywords']
    for col_name in cols_to_copy:
      our_summary[col_name] = ''
    our_summary[nameOfYourSummary] = ''
    recent_index = -1
  
  else:
    our_summary = pd.read_pickle(recent_path)

  for index, row in original_dataset.iloc[recent_index+1:].iterrows():
    paper = pd.Series(row).copy().to_frame().T
    generated_summary = summary_function(paper, **kwargs)
    # print(generated_summary)
    paper[nameOfYourSummary] = generated_summary


    #our_summary = our_summary.append(paper)
    our_summary = pd.concat([our_summary, paper])

    print(f"pickling index {index}")
    our_summary.to_pickle(f'{save_folder_path}checkpoint_{index}.pkl')
    # return our_summary

In [1]:
!rm -rf pickles
!mkdir pickles

In [ ]:
summarize_with_checkpoint(path = 'elife.zip', JSONfileName = 'test.json', summary_function = newLeadK, nameOfYourSummary = 'leadK_synonyms', save_folder_path = '/content/pickles/', k=5)

recent index is None
pickling index 0
pickling index 1
pickling index 2
pickling index 3
pickling index 4
pickling index 5
pickling index 6
pickling index 7
pickling index 8
pickling index 9
pickling index 10
pickling index 11
pickling index 12
pickling index 13
pickling index 14
pickling index 15
pickling index 16
pickling index 17
pickling index 18
pickling index 19
pickling index 20
pickling index 21
pickling index 22
pickling index 23
pickling index 24
pickling index 25
pickling index 26
pickling index 27
pickling index 28
pickling index 29
pickling index 30
pickling index 31
pickling index 32
pickling index 33
pickling index 34
pickling index 35
pickling index 36
pickling index 37
pickling index 38
pickling index 39
pickling index 40
pickling index 41
pickling index 42
pickling index 43
pickling index 44
pickling index 45
pickling index 46
pickling index 47
pickling index 48
pickling index 49
pickling index 50
pickling index 51
pickling index 52
pickling index 53
pickling index 54

In [ ]:
# get the last pickle file and dump the dataFrame in JSON format
def dumpLastPklToJson(path, jsonFileName, jsonFileStoringPath):
    # Get list of all pkl files in the directory
    pkl_files = glob.glob(os.path.join(path, 'checkpoint_*.pkl'))

    # If there are no pkl files, return
    if not pkl_files:
        print("No pkl files found")
        return

    # Sort the files by modification time
    pkl_files.sort(key=os.path.getmtime)

    # Get the latest file
    latest_pkl_file = pkl_files[-1]

    # Load data from the latest pkl file
    with open(latest_pkl_file, 'rb') as f:
        data = pickle.load(f)

    # Dump data to JSON file
    json_file_path = os.path.join(jsonFileStoringPath, jsonFileName)
    if isinstance(data, pd.DataFrame):
        # If data is a DataFrame, use pandas' to_json() method
        data.to_json(json_file_path, orient="records", lines=True)
    else:
        # If data is not a DataFrame, use the json module
        with open(json_file_path, 'w') as f:
            json.dump(data, f, indent=4)

    print(f"Data from {latest_pkl_file} has been dumped to {json_file_path}")


In [ ]:
dumpLastPklToJson("/content/pickles", "leadKSynonyms.json", "/content")

Data from /content/pickles/checkpoint_240.pkl has been dumped to /content/leadKSynonyms.json


In [ ]:
data = []
with open('/content/leadKSynonyms.json') as f:
    for line in f:
        data.append(json.loads(line))

dataF = pd.DataFrame(data)
dataFMySummary = dataF["leadK_synonyms"]
print(len(dataFMySummary[15]))

20


In [ ]:
dataF

,id,year,title,abstract,summary,keywords,leadK_synonyms,sections,headings
0,elife-35500-v1,2018,National and regional seasonal dynamics of all...,"[In temperate climates , winter deaths exceed ...","[In the USA , more deaths happen in the winter...",[epidemiology and global health],[It is well-established that death rates vary ...,[[It is well-established that death rates vary...,"[Introduction, Results, Discussion, Materials ..."
1,elife-48378-v2,2019,Complement and CD4+ T cells drive context-spec...,[Whether complement dysregulation directly con...,[Most people have likely experienced the disco...,"[microbiology and infectious disease, immunolo...",[Dysregulated complement activation is increas...,[[Dysregulated complement activation is increa...,"[Introduction, Results, Discussion, Materials ..."
2,elife-04494-v1,2015,Phenotypic complementation of genetic immunode...,[Variation in the presentation of hereditary i...,[The immune system protects an individual from...,"[microbiology and infectious disease, immunolo...","[HOIL-1 ( encode by the RBCK1 gene ) , HOIP ( ...","[[HOIL-1 ( encoded by the RBCK1 gene ) , HOIP ...","[Introduction, Results, Discussion, Materials ..."
3,elife-12352-v2,2016,Cascade of neural processing orchestrates cogn...,[Rapid and flexible interpretation of conflict...,[The brain adapts to control our behavior in d...,[neuroscience],[Flexible control of cognitive processes is fu...,[[Flexible control of cognitive processes is f...,"[Introduction, Results, Discussion, Materials ..."
4,elife-05413-v2,2015,Structural dynamics of myosin 5 during process...,[Myosin 5a is a dual-headed molecular motor th...,[Cells use motor proteins that to move organel...,[structural biology and molecular biophysics],[myosin 5a moves in a hand-over-hand fashion w...,[[Myosin 5a moves in a hand-over-hand fashion ...,"[Introduction, Results, Discussion, Materials ..."
...,...,...,...,...,...,...,...,...,...
1996,elife-49956-v2,2019,An N-terminal motif in NLR immune receptors is...,[The molecular codes underpinning the function...,"[Just like humans , plants get sick ., They ca...",[plant biology],[Plants have evolved intracellular immune rece...,[[Plants have evolved intracellular immune rec...,"[Introduction, Results, Discussion, Materials ..."
1997,elife-72294-v2,2022,External validation of a mobile clinical decis...,[Diarrheal illness is a leading cause of antib...,[Diarrhea is one of the most common illnesses ...,"[epidemiology and global health, medicine]",[diarrheal diseases remain a leading cause of ...,[[Diarrheal diseases remain a leading cause of...,"[Introduction, Materials and methods, Results,..."
1998,elife-43657-v1,2019,An atlas of polygenic risk score associations ...,[The age of large-scale genome-wide associatio...,[An individual’s risk of developing many disea...,"[tools and resources, genetics and genomics]",[Developing our understanding of how modifiabl...,[[Developing our understanding of how modifiab...,"[Introduction, Results, Discussion, Materials ..."
1999,elife-05608-v2,2015,Graded Ca2+/calmodulin-dependent coupling of v...,"[In the heart , reliable activation of Ca2+ re...","[To pump blood around the body , the muscle ce...",[structural biology and molecular biophysics],[L-type Ca2+ channels are composed of a pore-f...,[[L-type Ca2+ channels are composed of a pore-...,"[Introduction, Results, Discussion, Materials ..."
